## Ejercicios de ecuaciones de congruencias

In [1]:
# A) 3x ≡ 2 (mod 7)
#Método 1: con inverse_mod
x1 = (inverse_mod(3, 7) * 2) % 7
x1, (3*x1) % 7   # muestra la solución y verifica el residuo


(3, 2)

In [2]:
# 3x ≡ 2 (mod 7)
#Método 2: trabajando en Z/nZ
R = Zmod(7)
x1b = (R(2) / R(3))          # elemento en Z/7Z
x1b_lift = x1b.lift()        # representante entero en [0,6]
x1b, x1b_lift, (3*x1b_lift) % 7


(3, 3, 2)

In [4]:
# B) 5x+1≡13(mod23)

x1 = (inverse_mod(5,23) * 12) % 23
x1 , (5*x1) % 23       

(7, 12)

In [5]:
R = Zmod(23)
x2b = (R(13) - 1) / R(5)     # (13-1)/5 en Z/23Z
x2b_lift = x2b.lift()
x2b, x2b_lift, (5*x2b_lift + 1) % 23


(7, 7, 13)

# ¿Qué significa `x1b = (R(2) / R(3))  # elemento en Z/7Z`?

`R = Zmod(7)` crea el anillo $ \mathbb{Z}/7\mathbb{Z} $ (enteros módulo 7).  
`R(2)` es la **clase de residuo** $[2]$ en $ \mathbb{Z}/7\mathbb{Z} $ y `R(3)` es $[3]$.

El operador `/` en este anillo hace **división modular**, es decir **multiplicar por el inverso**:

$$
R(2)/R(3) \;=\; R(2)\cdot (R(3))^{-1}.
$$

Como $7$ es primo, $ \mathbb{Z}/7\mathbb{Z} $ es un **cuerpo** y todo no nulo tiene inverso. En particular, $[3]^{-1}=[5]$ porque $3\cdot5\equiv 1 \pmod{7}$.

Por eso:
$$
x1b = [2]\cdot[3]^{-1} = [2]\cdot[5] = [10] = [3] \pmod{7}.
$$

---

## Cosas útiles en Sage

```sage
R = Zmod(7)
x1b = R(2) / R(3)

x1b                    # elemento en Z/7Z (clase de residuo)
x1b.lift()             # representante entero canónico en [0,6] -> 3
(3*x1b == R(2))        # True: verifica que 3 * x1b = 2 en Z/7Z
R(3).inverse()         # inverso de 3 -> 5
R(2) * R(3)^(-1)       # misma idea que dividir
R(2) * inverse_mod(3,7)# equivalente usando inverse_mod


In [7]:
# B') ax≡b(mod m)
a = 9
b = 12
m = 15

x1 = (inverse_mod(a,m) * b) % m
x1 , (a*x1) % m       

ZeroDivisionError: inverse of Mod(9, 15) does not exist

In [8]:
R = Zmod(m)
x2b = (R(12)) / R(a)     # (13-1)/5 en Z/23Z
x2b_lift = x2b.lift()
x2b, x2b_lift, (a*x2b_lift) % m


ZeroDivisionError: inverse of Mod(9, 15) does not exist

# OJO!!! 
## Este abordaje de usar el inverse_mod(a,m) solo aplica cuando m es primo, pues en ese caso todas las clases modulo m tiene ivnerso, si m es compuesto no todos tiene inverso
## y este abordaje falla, como se ve más arriba.

# ¿Cuándo falla el método del inverso en congruencias modulares?

El “abordaje del inverso” falla cuando $a$ no es invertible módulo $n$, es decir, cuando $\gcd(a,n)\neq 1$.  
En $\mathbb{Z}/n\mathbb{Z}$ sólo se puede dividir por **unidades** (clases coprimas con $n$).

---

## Regla general para $a x \equiv b \pmod n$

Sea $d=\gcd(a,n)$.

1. Si $d \nmid b$ ⇒ **no hay solución**.
2. Si $d \mid b$ ⇒ **sí hay soluciones**. Reducí:
   $$
   a'=\frac{a}{d},\quad b'=\frac{b}{d},\quad n'=\frac{n}{d}.
   $$
   Como $\gcd(a',n')=1$, existe $a'^{-1}\pmod{n'}$ y
   $$
   x_0 \equiv a'^{-1}\,b' \pmod{n'}.
   $$
   Las **soluciones módulo $n$** son
   $$
   x \equiv x_0 \pmod{n'} \;\Longleftrightarrow\; x = x_0 + k\,n',\; k=0,1,\dots,d-1.
   $$
   (Hay $d$ clases distintas módulo $n$.)

---

## Ejemplos

**1) $6x\equiv 8 \pmod{14}$**  
$d=\gcd(6,14)=2$ y $2\mid 8$ ⇒ reducimos a
$$
3x\equiv 4 \pmod 7.
$$
Como $3^{-1}\equiv 5 \pmod 7$, entonces
$$
x_0 \equiv 5\cdot 4 \equiv 20 \equiv 6 \pmod 7.
$$
Soluciones módulo $14$:
$$
x \equiv 6,\,13 \pmod{14}\quad(\text{i.e. } x=6+7k,\;k=0,1).
$$

**2) $6x\equiv 7 \pmod{14}$**  
$d=2\nmid 7$ ⇒ **no tiene solución**.

---

## En SageMath (robusto también para $n$ compuesto)

```sage
def resolver_congruencia(a, b, n):
    d = gcd(a, n)
    if b % d != 0:
        return dict(tiene_solucion=False, mensaje="No hay solución", gcd=d)
    a1, b1, n1 = a//d, b//d, n//d
    x0 = (inverse_mod(a1, n1) * b1) % n1
    soluciones_mod_n = [(x0 + k*n1) % n for k in range(d)]
    return dict(tiene_solucion=True, gcd=d, x0_mod_n1=x0, modulo_reducido=n1,
                clases_distintas_mod_n=sorted(set(soluciones_mod_n)))

# Ejemplos
resolver_congruencia(6, 8, 14)   # -> solvable (dos clases mod 14)
resolver_congruencia(6, 7, 14)   # -> no solvable


In [18]:
def resolver_congruencia(a, b, n):
    d = gcd(a, n)
    if b % d != 0:
        return dict(tiene_solucion=False, mensaje="No hay solución", gcd=d)
    a1, b1, n1 = a//d, b//d, n//d
    x0 = (inverse_mod(a1, n1) * b1) % n1
    soluciones_mod_n = [(x0 + k*n1) % n for k in range(d)]
    return dict(tiene_solucion=True, gcd=d, x0_mod_n1=x0, modulo_reducido=n1,
                clases_distintas_mod_n=sorted(set(soluciones_mod_n)))


In [19]:
resolver_congruencia(9, 12, 15)   # -> solvable (dos clases mod 14)


{'tiene_solucion': True,
 'gcd': 3,
 'x0_mod_n1': 3,
 'modulo_reducido': 5,
 'clases_distintas_mod_n': [3, 8, 13]}

In [20]:
resolver_congruencia(6, 7, 14)   # -> no solvable

{'tiene_solucion': False, 'mensaje': 'No hay solución', 'gcd': 2}

In [21]:
resolver_congruencia(5,12,26)

{'tiene_solucion': True,
 'gcd': 1,
 'x0_mod_n1': 18,
 'modulo_reducido': 26,
 'clases_distintas_mod_n': [18]}

In [22]:
# B) 5x+1≡13(mod26)
a = 5
b = 12
m = 26

x1 = (inverse_mod(a,m) * b) % m
x1 , (a*x1) % m             

(18, 12)